In [ ]:
from prompt_graph.tasker import NodeTask, GraphTask
from prompt_graph.utils import seed_everything
from torchsummary import summary
from prompt_graph.utils import print_model_parameters
from prompt_graph.utils import get_args
from prompt_graph.data import load4node, load4graph, split_induced_graphs
import pickle
import random
import numpy as np
import os
import pandas as pd

/home/chenyizi/miniconda3/envs/Pyg2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You can unzip the Experiment.zip to get the induced_graph file or run to get the induced_graph by yourself

In [ ]:
def load_induced_graph(dataset_name, data, device):

    folder_path = "./Experiment/induced_graph/" + dataset_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    file_path = folder_path + "/induced_graph_min100_max300.pkl"
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            print("loading induced graph...")
            graphs_list = pickle.load(f)
            print("Done!!!")
    else:
        print("Begin split_induced_graphs.")
        split_induced_graphs(
            data, folder_path, device, smallest_size=100, largest_size=300
        )
        with open(file_path, "rb") as f:
            graphs_list = pickle.load(f)
    graphs_list = [graph.to(device) for graph in graphs_list]
    return graphs_list

set the parameters.

In [ ]:
class get_args:
    task = "NodeTask"  # 根据你的需求设置
    dataset_name = "Cora"
    device = 3
    gnn_type = "GCN"
    prompt_type = "GPF-plus"
    hid_dim = 128
    batch_size = 128
    epochs = 1000
    shot_num = 1
    pre_train_model_path = (
        "./Experiment/pre_trained_model/Cora/Edgepred_Gprompt.GCN.128hidden_dim.pth"
    )
    lr = 0.02
    decay = 2e-6
    num_layer = 2
    seed = 42

In [4]:
args = get_args()
seed_everything(args.seed)

In [ ]:
if args.downstream_task == "NodeTask":
    data, input_dim, output_dim = load4node(args.dataset_name)
    data = data.to(args.device)
    if args.prompt_type in ["Gprompt", "All-in-one", "GPF", "GPF-plus"]:
        graphs_list = load_induced_graph(args.dataset_name, data, args.device)
    else:
        graphs_list = None


if args.downstream_task == "GraphTask":
    input_dim, output_dim, dataset = load4graph(args.dataset_name)

if args.downstream_task == "NodeTask":
    tasker = NodeTask(
        pre_train_model_path=args.pre_train_model_path,
        dataset_name=args.dataset_name,
        num_layer=args.num_layer,
        gnn_type=args.gnn_type,
        hid_dim=args.hid_dim,
        prompt_type=args.prompt_type,
        epochs=args.epochs,
        shot_num=args.shot_num,
        device=args.device,
        lr=args.lr,
        wd=args.decay,
        batch_size=args.batch_size,
        data=data,
        input_dim=input_dim,
        output_dim=output_dim,
        graphs_list=graphs_list,
    )


if args.downstream_task == "GraphTask":
    tasker = GraphTask(
        pre_train_model_path=args.pre_train_model_path,
        dataset_name=args.dataset_name,
        num_layer=args.num_layer,
        gnn_type=args.gnn_type,
        hid_dim=args.hid_dim,
        prompt_type=args.prompt_type,
        epochs=args.epochs,
        shot_num=args.shot_num,
        device=args.device,
        lr=args.lr,
        wd=args.decay,
        batch_size=args.batch_size,
        dataset=dataset,
        input_dim=input_dim,
        output_dim=output_dim,
    )
pre_train_type = tasker.pre_train_type


_, test_acc, std_test_acc, f1, std_f1, roc, std_roc, _, _ = tasker.run()

print("Final Accuracy {:.4f}±{:.4f}(std)".format(test_acc, std_test_acc))
print("Final F1 {:.4f}±{:.4f}(std)".format(f1, std_f1))
print("Final AUROC {:.4f}±{:.4f}(std)".format(roc, std_roc))

Cora
loading induced graph...
Done!!!
GCN(
  (conv_layers): ModuleList(
    (0): GCNConv(1433, 128)
    (1): GCNConv(128, 128)
  )
)
Successfully loaded pre-trained weights!
idx_train tensor([1144,   46,  451, 1279, 1370, 1942, 1451], device='cuda:3')
true 1 tensor([1, 2, 5, 0, 6, 3, 4], device='cuda:3')
distinguishing the train dataset and test dataset...
Done!!!
prepare induce graph data is finished!
Epoch 001 |  Time(s) 0.2460 | Loss 1.9460  
Epoch 002 |  Time(s) 0.0048 | Loss 1.9451  
Epoch 003 |  Time(s) 0.0045 | Loss 1.9456  
Epoch 004 |  Time(s) 0.0045 | Loss 1.9437  
Epoch 005 |  Time(s) 0.0045 | Loss 1.9436  
Epoch 006 |  Time(s) 0.0047 | Loss 1.9419  
Epoch 007 |  Time(s) 0.0045 | Loss 1.9387  
Epoch 008 |  Time(s) 0.0043 | Loss 1.9318  
Epoch 009 |  Time(s) 0.0045 | Loss 1.9196  
Epoch 010 |  Time(s) 0.0046 | Loss 1.9170  
Epoch 011 |  Time(s) 0.0044 | Loss 1.9037  
Epoch 012 |  Time(s) 0.0045 | Loss 1.8871  
Epoch 013 |  Time(s) 0.0045 | Loss 1.8638  
Epoch 014 |  Time(s) 0